<a href="https://colab.research.google.com/github/bhadreshpsavani/UnderstandingNLP/blob/master/TFLiteExperiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers

     |████████████████████████████████| 1.3MB 2.8MB/s 
     |████████████████████████████████| 2.9MB 18.2MB/s 
     |████████████████████████████████| 890kB 29.5MB/s 
     |████████████████████████████████| 1.1MB 40.8MB/s 


In [2]:
import numpy as np
import tensorflow as tf
from transformers import TFDistilBertForQuestionAnswering, DistilBertTokenizer

In [19]:
model = TFDistilBertForQuestionAnswering.from_pretrained('distilbert-base-uncased-distilled-squad')

input_spec = tf.TensorSpec([1, 384], tf.int32)
model._set_inputs(input_spec, training=False)

print(model.inputs)
print(model.outputs)

Some layers from the model checkpoint at distilbert-base-uncased-distilled-squad were not used when initializing TFDistilBertForQuestionAnswering: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased-distilled-squad and are newly initialized: ['dropout_79']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


None
None


In [20]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased-distilled-squad')

In [21]:
question, text = "Who was Jim Henson?", "Jim Henson was a nice puppet"
input_dict = tokenizer(question, text, return_tensors='tf', padding=True, add_special_tokens=True, max_length=384, truncation=True)

In [22]:
outputs = model(input_dict)
outputs

(<tf.Tensor: shape=(1, 14), dtype=float32, numpy=
 array([[-4.7436023, -4.0272107, -7.136568 , -3.8197165, -7.084014 ,
         -7.5087566, -5.254179 ,  2.8774292, -2.9910977, -2.302865 ,
          6.1948557,  5.0000935,  2.2608106, -5.2538705]], dtype=float32)>,
 <tf.Tensor: shape=(1, 14), dtype=float32, numpy=
 array([[ 1.2521738 , -3.9068744 , -6.5817323 , -7.3439803 , -2.7099144 ,
         -5.304587  , -0.33467034, -4.213182  ,  0.38150564, -5.37238   ,
         -3.127     ,  0.69834507,  6.364172  , -0.33409446]],
       dtype=float32)>)

In [23]:
start_logits = outputs.start_logits
end_logits = outputs.end_logits
all_tokens = tokenizer.convert_ids_to_tokens(input_dict["input_ids"].numpy()[0])
answer = ' '.join(all_tokens[tf.math.argmax(start_logits, 1)[0] : tf.math.argmax(end_logits, 1)[0]+1])

AttributeError: ignored

In [15]:
from transformers import DistilBertTokenizer, TFDistilBertForQuestionAnswering
import tensorflow as tf
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = TFDistilBertForQuestionAnswering.from_pretrained('distilbert-base-uncased', return_dict=True)
question, text = "Who was Jim Henson?", "Jim Henson was a nice puppet"
input_dict = tokenizer(question, text, return_tensors='tf')
outputs = model(input_dict)
start_logits = outputs.start_logits
end_logits = outputs.end_logits
all_tokens = tokenizer.convert_ids_to_tokens(input_dict["input_ids"].numpy()[0])
answer = ' '.join(all_tokens[tf.math.argmax(start_logits, 1)[0] : tf.math.argmax(end_logits, 1)[0]+1])

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForQuestionAnswering: ['vocab_transform', 'vocab_layer_norm', 'vocab_projector', 'activation_13']
- This IS expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dropout_59', 'qa_outputs']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [40]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# For normal conversion:
converter.target_spec.supported_ops = [tf.lite.OpsSet.SELECT_TF_OPS]

# Below Two methods makes models size 4 time smaller
# For conversion with FP16 quantization:
# supports CPUs, GPUs
# converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
# converter.target_spec.supported_types = [tf.float16]
# converter.optimizations = [tf.lite.Optimize.DEFAULT]
# converter.experimental_new_converter = True

# For conversion with hybrid quantization:
# This only support CPU
# converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
# converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
# converter.experimental_new_converter = True

# another optimization is OPTIMIZE_FOR_LATENCY

In [41]:
tflite_model = converter.convert()
open("distilbert-squad-384.tflite", "wb").write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpgl8l9aec/assets


INFO:tensorflow:Assets written to: /tmp/tmpgl8l9aec/assets
INFO:absl:Using experimental converter: If you encountered a problem please file a bug. You can opt-out by setting experimental_new_converter=False


264105356

In [43]:
# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="distilbert-squad-384.tflite")
interpreter.allocate_tensors()

In [44]:
# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [45]:
input_details

[{'dtype': numpy.int32,
  'index': 0,
  'name': 'input_ids',
  'quantization': (0.0, 0),
  'quantization_parameters': {'quantized_dimension': 0,
   'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32)},
  'shape': array([3, 5], dtype=int32),
  'shape_signature': array([3, 5], dtype=int32),
  'sparsity_parameters': {}}]

In [46]:
output_details

[{'dtype': numpy.float32,
  'index': 634,
  'name': 'Identity',
  'quantization': (0.0, 0),
  'quantization_parameters': {'quantized_dimension': 0,
   'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32)},
  'shape': array([3, 5], dtype=int32),
  'shape_signature': array([3, 5], dtype=int32),
  'sparsity_parameters': {}},
 {'dtype': numpy.float32,
  'index': 636,
  'name': 'Identity_1',
  'quantization': (0.0, 0),
  'quantization_parameters': {'quantized_dimension': 0,
   'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32)},
  'shape': array([3, 5], dtype=int32),
  'shape_signature': array([3, 5], dtype=int32),
  'sparsity_parameters': {}}]

In [74]:
list(input_dict['input_ids'].numpy()[0])

[101,
 2040,
 2001,
 3958,
 27227,
 1029,
 102,
 3958,
 27227,
 2001,
 1037,
 3835,
 13997,
 102]

In [31]:
input_dict.keys()

dict_keys(['input_ids', 'attention_mask'])

In [75]:
# Test the model on random input data.
input_data1 = list(input_dict['input_ids'].numpy()[0])
input_data2 = list(input_dict['attention_mask'].numpy()[0])
inputs = []
inputs.append(input_data1)
inputs.append(input_data2)

In [76]:
interpreter.set_tensor(input_details[0]['index'], inputs)
interpreter.invoke()

ValueError: ignored

In [ ]:
outputs = model(input_dict)

In [ ]:
start_logits = outputs.start_logits
end_logits = outputs.end_logits
all_tokens = tokenizer.convert_ids_to_tokens(input_dict["input_ids"].numpy()[0])

In [ ]:
answer = ' '.join(all_tokens[tf.math.argmax(start_logits, 1)[0] : tf.math.argmax(end_logits, 1)[0]+1])

Reference:

* https://www.tensorflow.org/lite/performance/post_training_quantization
* https://colab.research.google.com/github/tensorflow/examples/blob/master/courses/udacity_intro_to_tensorflow_lite/tflite_c02_transfer_learning.ipynb
* https://colab.research.google.com/github/tensorflow/examples/blob/master/courses/udacity_intro_to_tensorflow_lite/tflite_c04_exercise_convert_model_to_tflite_solution.ipynb

In [ ]:
""